In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.autograd import Variable
import numpy as np
import pandas as pd
from mash_data_utils import *

c:\users\lukaa\appdata\local\continuum\anaconda3\envs\torch\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 1. Dataset preprocessing

In [2]:
dataset = pd.read_csv('mushrooms.csv')

In [3]:
dataset.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [4]:
dataset = prepare_dataset(dataset)

In [5]:
dataset.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [6]:
X, y = labels_vs_features(dataset)

In [7]:
X_train, X_test, y_train, y_test =  data_splitter(X, y)

X_train size --> (6499, 22)
y_train size --> (6499,)
X_test size --> (1625, 22)
y_test size --> (1625,)


In [8]:
y_train = y_train.reshape(-1, 1)

In [9]:
def accuracy(preds, y_true):
    '''
        Use this function to calculate accuracy of a model for binary classification task.
    '''
    correct = 0
    assert len(preds) == len(y_true)
    for i in range(len(preds)):
        if preds[i] >= 0.5:
            if y_true[i] == 1:
                correct += 1
        else:
            if y_true[i] == 0:
                correct += 1
    
    return correct / len(preds)

### 2. Creating PyTorch FCNN

In [14]:
class FullyConnectedNN(nn.Module):
    
    def __init__(self, num_of_features):
        super(FullyConnectedNN, self).__init__()
        
        self.fc_1 = nn.Linear(num_of_features, 100)
        self.fc_2 =  nn.Linear(100, 1)
    
    def forward(self, X):
        l1 = f.relu(self.fc_1(X))
        output = f.sigmoid(self.fc_2(l1))
        return output

In [15]:
#hyperparams
features = 22
epochs = 150
batch_size = 128
learning_rate = 0.01

In [16]:
net = FullyConnectedNN(features)

In [17]:
#Execute this cell if you have GPU that is cuda supported.
net.cuda()

FullyConnectedNN(
  (fc_1): Linear(in_features=22, out_features=100)
  (fc_2): Linear(in_features=100, out_features=1)
)

#### Defining a loss function

In [18]:
criterion = nn.BCELoss()

#### Defining an optimizer

In [19]:
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

#### Training loop

In [20]:
for epoch in range(epochs):
    
    epoch_accuracy = []
    epoch_loss = []
    for i in range(len(X_train) // batch_size):
        starting_id = i * batch_size
        ending_id = starting_id + batch_size
        
        X_batch = Variable(torch.from_numpy(np.float32(X_train[starting_id:ending_id])).cuda())
        y_batch = Variable(torch.from_numpy(np.float32(y_train[starting_id:ending_id])).cuda())
        
        optimizer.zero_grad() #reset net to 0 grads
        predictions = net(X_batch)
        epoch_accuracy.append(accuracy(predictions.cpu().data.numpy(), y_train[starting_id:ending_id]))
        loss = criterion(predictions, y_batch)
        epoch_loss.append(loss.cpu().data.numpy())
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 10 == 0:
        print('Epoch: {}/{}'.format(epoch+1, epochs), 
              ' Epoch accuracy: {}'.format(np.mean(epoch_accuracy)), 
              ' Epoch loss: {}'.format(np.mean(epoch_loss)))

Epoch: 10/150  Epoch accuracy: 0.88578125  Epoch loss: 0.30252063274383545
Epoch: 20/150  Epoch accuracy: 0.91171875  Epoch loss: 0.24167266488075256
Epoch: 30/150  Epoch accuracy: 0.929375  Epoch loss: 0.201266348361969
Epoch: 40/150  Epoch accuracy: 0.94125  Epoch loss: 0.16903968155384064
Epoch: 50/150  Epoch accuracy: 0.9540625  Epoch loss: 0.14341074228286743
Epoch: 60/150  Epoch accuracy: 0.96359375  Epoch loss: 0.12336785346269608
Epoch: 70/150  Epoch accuracy: 0.96921875  Epoch loss: 0.10742653161287308
Epoch: 80/150  Epoch accuracy: 0.97359375  Epoch loss: 0.0945136770606041
Epoch: 90/150  Epoch accuracy: 0.97609375  Epoch loss: 0.08381020277738571
Epoch: 100/150  Epoch accuracy: 0.9790625  Epoch loss: 0.07482398301362991
Epoch: 110/150  Epoch accuracy: 0.9815625  Epoch loss: 0.06725502759218216
Epoch: 120/150  Epoch accuracy: 0.9846875  Epoch loss: 0.06079068034887314
Epoch: 130/150  Epoch accuracy: 0.985625  Epoch loss: 0.05520924925804138
Epoch: 140/150  Epoch accuracy: 0.9

#### Testing

In [21]:
test_accuracy = []

for i in range(len(X_test) // batch_size):
    starting_id = i * batch_size
    ending_id = starting_id + batch_size

    X_batch = Variable(torch.from_numpy(np.float32(X_test[starting_id:ending_id])).cuda())
    y_batch = Variable(torch.from_numpy(np.float32(y_test[starting_id:ending_id])).cuda())

    predictions = net(X_batch)

    test_accuracy.append(accuracy(predictions.cpu().data.numpy(), y_test[starting_id:ending_id]))

In [22]:
np.mean(test_accuracy)

0.986328125

In [23]:
#Saving pytorch model
torch.save(net.state_dict(), 'mush_save.pkl')